### Getting deeper into Agents

### Converstaional ReAct Agents
The agent is designed for use in conversational settings. It incorporates the ReAct framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [17]:
from langchain.agents import create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo-0125', temperature = 0)

In [11]:
tools = load_tools(["llm-math"], llm=llm)

In [19]:
memory = ConversationBufferMemory(memory_key = "chat_history")

conversational_agent = initialize_agent(
    agent = "conversational-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3,
    memory = memory,
)

/var/folders/9y/bw4br9h516s_4pbmq21pjhl40000gn/T/ipykernel_3457/368169611.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  conversational_agent = initialize_agent(


In [21]:
output_1 = conversational_agent.run("Add 23 to 77 and tell me the result")
output_2 = conversational_agent.run("Add 10 to the previous result")

/var/folders/9y/bw4br9h516s_4pbmq21pjhl40000gn/T/ipykernel_3457/684693847.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_1 = conversational_agent.run("Add 23 to 77 and tell me the result")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 23 to 77
Observation: Answer: 100
Thought:Do I need to use a tool? No
AI: The result of adding 23 to 77 is 100.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 100 to 10
Observation: Answer: 110
Thought:Do I need to use a tool? No
AI: The result of adding 10 to the previous result of 100 is 110.

> Finished chain.


### ReAct Docstore

In [30]:
# !pip install wikipedia

In [40]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import initialize_agent, Tool
from langchain_openai import OpenAI

In [83]:
docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(name = "Search", func = docstore.search, description = "useful for when you need to search wikipedia"),
    Tool(name = "Lookup", func = docstore.lookup, description = "useful for when you need to look up a term in wikipedia")
]

llm = OpenAI(temperature = 0)

docstore_agent = initialize_agent(
    tools,
    llm,
    agent = "react-docstore",
    verbose = True,
    handling_parsing_errors = True,
    max_iterations = 8
)

In [58]:
docstore_agent.run("Tell me a few key things about Cristiano Ronaldo")



> Entering new AgentExecutor chain...
Thought: I need to search Cristiano Ronaldo and find key information about him.
Action: Search[Cristiano Ronaldo]
Observation: Cristiano Ronaldo dos Santos Aveiro  (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu] ; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both the Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won numerous individual accolades throughout his career, such as five Ballon d'Or awards, a record three UEFA Men's Player of the Year Awards, four European Golden Shoes, and was named five times the world's best player by FIFA, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42

"Cristiano Ronaldo is a Portuguese professional footballer, has won numerous individual accolades and holds multiple records, has played for multiple clubs, is the most-capped male player in history and the all-time top male goalscorer, and is one of the world's most marketable and famous athletes with a large social media following and a net worth of over $1 billion."

In [60]:
docstore_agent.invoke("Tell me a few key things about Mahatma Gandhi")



> Entering new AgentExecutor chain...
Thought: I need to search Mahatma Gandhi and find key information about him.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit, meaning great-souled or venerable), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London and was called to the bar at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, Gandhi moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to live in South Africa for 21 

{'input': 'Tell me a few key things about Mahatma Gandhi',
 'output': 'Indian lawyer, anti-colonial nationalist, political ethicist, nonviolent resistance, Father of the Nation, Gandhi Jayanti, nonviolent resistance, civil disobedience, assassinated by militant Hindu nationalist'}

### Docstore in LangChain is essentially a storage system of documents

### Self-ask with Search
This agent utilizes the intermediate answer tool for self asking questions

#### The Self-Ask approach allows for an LLM to give answers to questions it was not explicitly trained on. The model might not have the direct answer to a question, but answers to sub-questions will exist in the LLM dataset. It achieves by obtaining Intermediate answers

In [91]:
!pip install --upgrade --quiet duckduckgo-search

In [198]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func = search.run,
        name = "Intermediate Answer",
        description = "useful for when you need to search the internet for information"
    )
]


agent = initialize_agent(
    tools,
    llm = OpenAI(model_name = "gpt-3.5-turbo-instruct", temperature = 0.9) ,
    agent = "self-ask-with-search",
    verbose = True,
    handle_parsing_errors = True
)

In [204]:
print(agent.invoke("Question: Who was the President of the United States when the iPhone was first released?"))



> Entering new AgentExecutor chain...
Could not parse output:  No.
Final answer: The President of the United States when the iPhone was first released was George W. Bush.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Intermediate answer: Invalid or incomplete response
Could not parse output: 
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Intermediate answer: Invalid or incomplete response
So the final answer is: George W. Bush

> Finished chain.
{'input': 'Question: Who was the President of the United States when the iPhone was first released?', 'output': 'George W. Bush'}


### Usually, should ahve used the code with temperature = 0, but there was some issue with the model being used, hence to get the final output, increased temperature to 0.9.

In [219]:
print(agent.invoke("Question: Who was the President of the US when the world trade center, the Pentagon was attacked?"))



> Entering new AgentExecutor chain...
Could not parse output:  No. The answer can be found in the initial question.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Intermediate answer: Invalid or incomplete response
So the final answer is: The question needs to be rephrased to get an accurate answer.

> Finished chain.
{'input': 'Question: Who was the President of the US when the world trade center, the Pentagon was attacked?', 'output': 'The question needs to be rephrased to get an accurate answer.'}
